# convnet 中間出力画像の作成

入力画像ファイルと出力先ディレクトリを指定して VGG16 のすべてのフィルタごとの反応画像を出力します。

実行前に以下のライブラリを導入する必要があります。

```
$ apt install -y libsm6 libxext6 libfontconfig1 libxrender1
$ pip install opencv-python keras numpy
```

In [22]:
input_file = "kawasemi.jpg"
output_dir = "filtered-images_vgg16"

## 画像の読み込み

入力画像を読み込んで、Keras に付属の ImageNet 学習済み VGG16 と適合するように前処理をしたテンソルとして保持します。

In [23]:
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
import numpy as np

img = image.load_img(input_file, target_size=(224, 224))
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)
img = preprocess_input(img)
print("IMAGE: %s" % str(img.shape)) 

IMAGE: (1, 224, 224, 3)


次に、VGG16 のモデルから畳み込み層のレイヤーだけを取り出して活性化モデルを作成します。

In [24]:
from keras import models
from keras.applications.vgg16 import VGG16

model = VGG16(weights="imagenet")
layers = model.layers[0:19]
layer_outputs = [layer.output for layer in layers]
activation_model = models.Model(inputs=model.input, outputs=layer_outputs)
# activation_model.summary()

activations = activation_model.predict(img)
# for i, activation in enumerate(activations):
#   print("%2d: %s" % (i, str(activation.shape)))

ResourceExhaustedError: OOM when allocating tensor with shape[4096] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: fc1_6/bias/Assign = Assign[T=DT_FLOAT, _grappler_relax_allocator_constraints=true, use_locking=true, validate_shape=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](fc1_6/bias, fc1_6/Const)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'fc1_6/bias/Assign', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.5/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start

## フィルタ強度画像の作成

レイヤーの各フィルタを通過した画像がどの部分を特徴として強調されているかを可視化する画像を作成します。

ヒートマップのように表示されるようにカラーマップを使用しています。

In [ ]:
import os, math, cv2
from PIL import Image
from matplotlib.cm import get_cmap
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

if not os.path.isdir(output_dir):
  os.makedirs(output_dir)

cm = get_cmap("inferno")

for i, (activation, layer) in enumerate(zip(activations, layers)):
  print(activation, activation.shape)
  num_of_images = activation.shape[3]
  output_file = os.path.join(output_dir, "%s.png" % layer.name)
  acts = (cm(activation[0, :, :] / np.max(activation[0])) * 255).astype(np.int8).transpose([2, 0, 1, 3])
  rows = math.ceil(math.sqrt(num_of_images))
  cols = math.ceil(num_of_images / rows)
  vstack = []
  spacing = 1
  for y in range(0, cols):
    size = min(rows, num_of_images - y * cols)
    hstack = []
    for x in range(0, rows):
      if x is not 0:
        hstack.append(np.zeros((acts[0].shape[0], spacing, 4), dtype=np.int8))
      i = y * cols + x
      if i < len(acts):
        hstack.append(acts[i])
      else:
        hstack.append(np.zeros(acts[0].shape, dtype=np.int8))
    row = cv2.hconcat(hstack)
    vstack.append(row)
    if y + 1 < cols:
      vstack.append(np.zeros((spacing, row.shape[1], 4), dtype=np.int8))
  array = cv2.vconcat(vstack)
  image = Image.fromarray(array, "RGBA")
  image.save(output_file)
  print("%s -> %dx%d write to: %s %s" % (str(acts.shape), cols, rows, output_file, str(array.shape)))

plt.axis("off")
plt.imshow(image)